In [1]:
import urllib.request
import cv2
import numpy as np
import time
import ssl
import os
import time
import pandas as pd
import PIL as pillow
from PIL import Image
import matplotlib.pyplot as plt
from skimage import io
from skimage.transform import rescale, resize, downscale_local_mean
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf

# For some reason tf gpu gives an error when I don't run the following
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
  except RuntimeError as e:
    print(e)

In [2]:
train_path = "C:/Users/svabh/Documents/Python Scripts/Facial Recognition/train"
test_path = "C:/Users/svabh/Documents/Python Scripts/Facial Recognition/faces/test"
rootdir = "C:/Users/svabh/Documents/Python Scripts//Facial Recognition/faces"

In [3]:
filenamesDf = pd.read_csv('dataset.csv')

In [4]:
data = []
for value in filenamesDf['Filename']:
    img = io.imread(value.replace(".", rootdir, 1))
    img = resize(img, (60, 60),
                       anti_aliasing=True)
    data.append(img)
data = np.array(data)

In [5]:
image_shape = (60,60,3)

In [6]:

x_train, x_valid, y_train, y_valid= train_test_split(
    data, filenamesDf['Label'], test_size=.1, shuffle = True,)

In [7]:
image_gen = ImageDataGenerator(rotation_range=20,                               width_shift_range=0.10,
                               height_shift_range=0.10,
                               rescale=1/255,
                               shear_range=0.1,
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, 
                               fill_mode='nearest' 
                              )

In [8]:
train_image_gen = image_gen.flow(x_train, y_train, batch_size = 16)


In [9]:
test_image_gen = image_gen.flow(x_valid, y_valid, batch_size = 16)

In [10]:
x_gen,y_gen = train_image_gen.next()
y_gen

array([0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0], dtype=int64)

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping

In [12]:
model= Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape= image_shape),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape= image_shape),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    #20 is the number of outputs
    Dense(1, activation='sigmoid')  
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [14]:

model.fit(np.array(x_train), np.array(y_train), batch_size=16,
    epochs=32,verbose=2,
    validation_data=(np.array(x_valid),np.array(y_valid)), callbacks = [early_stop],
)

Train on 5656 samples, validate on 629 samples
Epoch 1/32
5656/5656 - 3s - loss: 0.0619 - accuracy: 0.9752 - val_loss: 1.4819e-08 - val_accuracy: 1.0000
Epoch 2/32
5656/5656 - 1s - loss: 0.0092 - accuracy: 0.9986 - val_loss: 0.0016 - val_accuracy: 0.9984
Epoch 3/32
5656/5656 - 1s - loss: 0.0047 - accuracy: 0.9993 - val_loss: 0.0035 - val_accuracy: 0.9984


In [253]:

scor = model.evaluate( np.array(x_valid),  np.array(y_valid), verbose=0)

print('test los {:.4f}'.format(scor[0]))
print('test acc {:.4f}'.format(scor[1]))

test los 0.0001
test acc 1.0000


In [254]:
pred_probabilities = model.predict_classes(x_valid)

In [255]:
pred_probabilities

array([[0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
    

In [256]:
predictions = pred_probabilities > 0.5

In [257]:
y_valid

4637    0
3324    1
1532    1
4895    0
4323    0
       ..
521     1
3703    0
4102    0
3723    0
351     1
Name: Label, Length: 629, dtype: int64

In [258]:
y_valid.shape

(629,)

In [259]:
predictions

array([[False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [

In [260]:
from sklearn.metrics import classification_report,confusion_matrix

In [261]:
print(classification_report(y_valid,predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       278
           1       1.00      1.00      1.00       351

    accuracy                           1.00       629
   macro avg       1.00      1.00      1.00       629
weighted avg       1.00      1.00      1.00       629



In [262]:
predictions

array([[False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [

In [263]:
image = io.imread('./Svabhu/Svabhu44.jpg'.replace(".", rootdir, 1))
image = resize(image, (60, 60), anti_aliasing=True)

In [264]:
model.predict_classes(image.reshape(1,60,60,3))

array([[0]])

In [265]:
model.save('faceRecognize.h5')

In [266]:
from tensorflow import keras
model = keras.models.load_model('faceRecognize.h5')